# Multiplicity determination with Scikit-learn classifiers

Here we test out scikit-learn classification models for multiplicity reconstruction with the three main features number of hits, number of clusters, and total deposited energy.

In [1]:
import numpy as np
import pandas as pd
import time
import multiprocessing
import functools
from joblib import Parallel, delayed

import sklearn
from sklearn import *
from sklearn.experimental import enable_hist_gradient_boosting

import sys

sys.path.append("..")
from helpers import filename_for

Welcome to JupyROOT 6.16/00


Load ALL the classification models from scikit-learn.
Note that some models are very slow to train with large datasets or crash outright, so we give them a reduced number of (shuffled) rows to learn.
Note that `n_jobs=1` is used, as parallelism is introduced later.

In [2]:
models = [
    ("AdaBoostClassifier", sklearn.ensemble.AdaBoostClassifier(), "fast"),
    ("BaggingClassifier", sklearn.ensemble.BaggingClassifier(n_jobs=1), "fast"),
    ("BernoulliNB", sklearn.naive_bayes.BernoulliNB(), "fast"),
    ("CalibratedClassifierCV", sklearn.calibration.CalibratedClassifierCV(cv=5), "slow"),
    ("ComplementNB", sklearn.naive_bayes.ComplementNB(), "fast"),
    ("DecisionTreeClassifier", sklearn.tree.DecisionTreeClassifier(), "fast"),
    ("ExtraTreeClassifier", sklearn.tree.ExtraTreeClassifier(), "fast"),
    ("ExtraTreesClassifier", sklearn.ensemble.ExtraTreesClassifier(n_estimators=100, n_jobs=1), "medi"),
    ("GaussianNB", sklearn.naive_bayes.GaussianNB(), "fast"),
    # crashes ('GaussianProcessClassifier', sklearn.gaussian_process.GaussianProcessClassifier(), 'slow'),
    ("GradientBoostingClassifier", sklearn.ensemble.GradientBoostingClassifier(), "slow"),
    ("HistGradientBoostingClassifier", sklearn.ensemble.HistGradientBoostingClassifier(), "slow"),
    ("KNeighborsClassifier", sklearn.neighbors.KNeighborsClassifier(n_jobs=1), "fast"),
    # ('LabelPropagation', sklearn.semi_supervised.LabelPropagation(), 'slow'),  # requires too much memory to train with larger datasets
    # ('LabelSpreading', sklearn.semi_supervised.LabelSpreading(), 'slow'),  # bit slow
    ("LinearDiscriminantAnalysis", sklearn.discriminant_analysis.LinearDiscriminantAnalysis(), "fast"),
    ("LinearSVC", sklearn.svm.LinearSVC(max_iter=20000), "slow"),  # slow with unscaled data
    (
        "LogisticRegression",
        sklearn.linear_model.LogisticRegression(solver="lbfgs", multi_class="auto", max_iter=20000),
        "slow",
    ),  # slow with unscaled data
    (
        "LogisticRegressionCV",
        sklearn.linear_model.LogisticRegressionCV(cv=5, solver="lbfgs", multi_class="auto", max_iter=20000),
        "slow",
    ),
    ("MLPClassifier", sklearn.neural_network.MLPClassifier(), "slow"),
    ("MultinomialNB", sklearn.naive_bayes.MultinomialNB(), "fast"),
    ("NearestCentroid", sklearn.neighbors.NearestCentroid(), "fast"),
    # nu infeasible ('NuSVC', sklearn.svm.NuSVC(), 'fast'),
    (
        "PassiveAggressiveClassifier",
        sklearn.linear_model.PassiveAggressiveClassifier(max_iter=1000, tol=1e-3, n_jobs=1),
        "fast",
    ),
    ("Perceptron", sklearn.linear_model.Perceptron(n_jobs=-1), "fast"),
    ("QuadraticDiscriminantAnalysis", sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis(), "fast"),
    (
        "RadiusNeighborsClassifier",
        sklearn.neighbors.RadiusNeighborsClassifier(radius=2, outlier_label=0, n_jobs=1),
        "fast",
    ),
    ("RandomForestClassifier", sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=1), "medi"),
    ("RidgeClassifier", sklearn.linear_model.RidgeClassifier(), "fast"),
    ("RidgeClassifierCV", sklearn.linear_model.RidgeClassifierCV(), "fast"),
    ("SGDClassifier", sklearn.linear_model.SGDClassifier(max_iter=1000, tol=1e-3, n_jobs=1), "medi"),
    ("SVC", sklearn.svm.SVC(gamma="scale"), "slow"),
]

Some models only work with properly scaled data, so we prepare ALL available scalers.

In [3]:
scalers = [
    # ('Unscaled data', ),
    ("standard scaling", sklearn.preprocessing.StandardScaler()),
    ("min-max scaling", sklearn.preprocessing.MinMaxScaler()),
    ("max-abs scaling", sklearn.preprocessing.MaxAbsScaler()),
    ("robust scaling", sklearn.preprocessing.RobustScaler(quantile_range=(25, 75))),
    ("power transformation (Yeo-Johnson)", sklearn.preprocessing.PowerTransformer(method="yeo-johnson")),
    # ('power transformation (Box-Cox)', sklearn.preprocessing.PowerTransformer(method='box-cox')), # 'strictly zero' meh.
    ("quantile transformation (gaussian pdf)", sklearn.preprocessing.QuantileTransformer(output_distribution="normal")),
    ("quantile transformation (uniform pdf)", sklearn.preprocessing.QuantileTransformer(output_distribution="uniform")),
    ("sample-wise L2 normalizing", sklearn.preprocessing.Normalizer()),
]

Train and Test data is passed through the scalers, including the "unscaled" scaler.

In [4]:
files = [filename_for(15, 30, 600, 500, n, "inclxx", s, "trifeature.pkl") for n in [1, 2, 3, 4] for s in range(20)]
dfs = [pd.read_pickle(file) for file in files]
data = pd.concat(dfs, ignore_index=True).sample(frac=1)
print(data.shape)

(800000, 6)


In [5]:
msk = np.random.rand(len(data)) < 0.8
msk[0] = True
msk[1] = False
data.loc[0] = data.loc[1] = [0, 0, 0, 0, 0, 0]

traindata = data[msk]
testdata = data[~msk]

print(traindata.shape)
print(testdata.shape)

(639665, 6)
(160335, 6)


In [6]:
trainscaled = [("Unscaled data", traindata[["nHits", "nClus", "Edep"]], testdata[["nHits", "nClus", "Edep"]],)] + [
    (
        sname,
        scaler.fit_transform(traindata[["nHits", "nClus", "Edep"]]),
        scaler.transform(testdata[["nHits", "nClus", "Edep"]]),
    )
    for sname, scaler in scalers
]

Run all model/scaler combinations, in parallel. Note that we use timeouts per task, as setting at timeout in joblib will throw everything.

In [7]:
def with_timeout(timeout):
    def decorator(decorated):
        @functools.wraps(decorated)
        def inner(*args, **kwargs):
            pool = multiprocessing.pool.ThreadPool(1)
            async_result = pool.apply_async(decorated, args, kwargs)
            try:
                return async_result.get(timeout)
            except multiprocessing.TimeoutError:
                return

        return inner

    return decorator

In [8]:
MEDIHEAD = 500000
SLOWHEAD = 250000
label = "nPN"

y_train_fast = traindata[[label]].values.ravel()
y_train_medi = traindata.head(MEDIHEAD)[[label]].values.ravel()
y_train_slow = traindata.head(SLOWHEAD)[[label]].values.ravel()
y_test = testdata[[label]].values.ravel()


@with_timeout(1200*2)
def train_model(mname, modelorg, speed, sname, x_train, x_test):
    # These get killed without error?
    if mname == "RadiusNeighborsClassifier" and sname != "Unscaled data":
        return (mname, sname, np.NaN, speed, np.NaN, "Skipped")
    try:
        model = sklearn.base.clone(modelorg)
        start = time.time()
        if speed == "slow":
            model.fit(x_train[0:SLOWHEAD], y_train_slow)
        elif speed == "medi":
            model.fit(x_train[0:MEDIHEAD], y_train_medi)
        elif speed == "fast":
            model.fit(x_train, y_train_fast)
        end = time.time()

        y_pred = model.predict(x_test)
        y_true = y_test

        bac = sklearn.metrics.balanced_accuracy_score(y_true, y_pred)
        return (mname, sname, bac, speed, (end - start), "ok")
    except Exception as err:
        return (mname, sname, np.NaN, speed, np.NaN, err)


def train_model_wrap(mname, modelorg, speed, sname, x_train, x_test):
    ret = train_model(mname, modelorg, speed, sname, x_train, x_test)
    if ret:
        return ret
    else:
        return (mname, sname, np.NaN, speed, np.NaN, "Timeout")

In [9]:
results = Parallel(n_jobs=30, backend="multiprocessing")(
    delayed(train_model_wrap)(mname, modelorg, speed, sname, x_train, x_test)
    for sname, x_train, x_test in trainscaled
    for mname, modelorg, speed in models
)

/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:715: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:715: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains class

In [10]:
resultsdf = pd.DataFrame(results)
pd.options.display.max_rows = 999
resultsdf.columns = ["Model", "Scaler", "BAC", "Speed", "Time", "Status"]
resultsdf.sort_values(by=["BAC", "Time", "Model"], ascending=[False, True, True], inplace=True)
resultsdf.style.hide_index().format({"BAC": "{:.2%}", "Time": "{:.2f}"}).bar(subset=["BAC"], color="lightgreen").bar(
    subset=["Time"], color="lightblue"
)

Model,Scaler,BAC,Speed,Time,Status
MLPClassifier,min-max scaling,72.36%,slow,759.77,ok
MLPClassifier,power transformation (Yeo-Johnson),72.34%,slow,408.54,ok
MLPClassifier,standard scaling,72.31%,slow,367.68,ok
MLPClassifier,max-abs scaling,72.31%,slow,1945.61,ok
MLPClassifier,robust scaling,72.31%,slow,710.88,ok
MLPClassifier,quantile transformation (uniform pdf),72.29%,slow,1945.56,ok
GradientBoostingClassifier,quantile transformation (uniform pdf),72.17%,slow,1660.55,ok
GradientBoostingClassifier,standard scaling,72.17%,slow,820.42,ok
GradientBoostingClassifier,power transformation (Yeo-Johnson),72.17%,slow,1230.63,ok
GradientBoostingClassifier,Unscaled data,72.17%,slow,881.20,ok
